In [1]:
# import modules
import pandas as pd
import mysql.connector
import hashlib


In [2]:
# read values from Users.csv
columns = ['email', 'password']
data = pd.read_csv('Users.csv', names=columns)
col1 = data['email'].values
col2 = data['password'].values


In [3]:
# connect to database
conn = mysql.connector.connect(host='localhost', user='root', passwd='admin')
cursor = conn.cursor()
# create database
cursor.execute('CREATE DATABASE IF NOT EXISTS phase2')

conn = mysql.connector.connect(
    host='localhost', user='root', passwd='admin', database='phase2')
cursor = conn.cursor()


# create tables
# Users
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Users (email VARCHAR(50), password VARCHAR(40), PRIMARY KEY(email))'
)
# Buyers
cursor.execute(
    "CREATE TABLE IF NOT EXISTS Buyers (email VARCHAR(50), first_name VARCHAR(25), last_name VARCHAR(25), gender VARCHAR(25), age INT, home_address_id VARCHAR(40), billing_address_id VARCHAR(40), PRIMARY KEY(email), FOREIGN KEY (email) REFERENCES Users(email))"
)
# Credit_Cards
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Credit_Cards(credit_card_num VARCHAR(20), card_code VARCHAR(5), expire_month VARCHAR(2), expire_year VARCHAR(4), card_type VARCHAR(20), Owner_email VARCHAR(50) NOT NULL, PRIMARY KEY(credit_card_num), FOREIGN KEY(Owner_email) REFERENCES Buyers(email) ON DELETE CASCADE)'
)
# Address
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Address (address_ID VARCHAR(40), zipcode INT, street_num VARCHAR(6), street_name VARCHAR(50), PRIMARY KEY(address_ID))'
)
# Zipcode_Info
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Zipcode_Info (zipcode INT, city VARCHAR(30), state_ID VARCHAR(2), population INT, density FLOAT, county_name VARCHAR(20), timezone VARCHAR(30), PRIMARY KEY (zipcode))'
)
# Sellers
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Sellers (email VARCHAR(50), routing_number VARCHAR(20), account_number VARCHAR(10), balance INT, PRIMARY KEY (email), FOREIGN KEY (email) REFERENCES Users(email))'
)
# Local_Vendors
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Local_Vendors (email VARCHAR(50), Bussiness_Name VARCHAR(50), Bussiness_Address_ID VARCHAR(40), Customer_Service_Number VARCHAR(20), PRIMARY KEY(email), FOREIGN KEY (email) REFERENCES Sellers(email) ON DELETE CASCADE)'
)
# Categories
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Categories (parent_category VARCHAR(50), category_name VARCHAR(50), PRIMARY KEY (category_name))'
)
# Product_Listings
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Product_Listings (Seller_Email VARCHAR(50), Listing_ID INT, Category VARCHAR(50), Title VARCHAR(100), Product_Name VARCHAR(100), Product_Description VARCHAR(100), Price VARCHAR(20), Quantity INT, Active_period INT, Status INT, PRIMARY KEY (Seller_Email, Listing_ID), FOREIGN KEY (Seller_Email) REFERENCES Sellers(email))'
)
# Orders
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Orders (Transaction_ID INT, Seller_Email VARCHAR(50), Listing_ID INT, Buyer_Email VARCHAR(50), Date VARCHAR(20), Quantity INT, Payment INT, PRIMARY KEY (Transaction_ID), FOREIGN KEY (Seller_Email) REFERENCES Sellers(email), FOREIGN KEY(Buyer_Email) REFERENCES Buyers(email))'
)
# Reviews
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Reviews (Buyer_Email VARCHAR(50), Seller_Email VARCHAR(50), Listing_ID INT, Review_Desc VARCHAR(50), PRIMARY KEY (Buyer_Email,Seller_Email, Listing_ID), FOREIGN KEY (Buyer_Email) REFERENCES Buyers(email), FOREIGN KEY (Seller_Email, Listing_ID) REFERENCES Product_Listings(Seller_email, Listing_ID))'
)
# Rating
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Rating (Buyer_Email VARCHAR(50), Seller_Email VARCHAR(50), Date VARCHAR(20), Rating INT, Rating_Desc VARCHAR(30), PRIMARY KEY(Buyer_Email,Seller_Email,Date), FOREIGN KEY (Buyer_Email) REFERENCES Buyers(email), FOREIGN KEY (Seller_Email) REFERENCES Sellers(email))'
)
# Shopping_Cart
cursor.execute(
    'CREATE TABLE IF NOT EXISTS Shopping_Cart (Shopping_Cart_ID INT NOT NULL AUTO_INCREMENT, Buyer_Email VARCHAR(50), Listing_ID INT, PRIMARY KEY (Shopping_Cart_ID))'
)


In [4]:
# import Users data from Users.csv to table Users
'''conn = mysql.connector.connect(
    host='localhost', user='root', passwd='admin', database='phase2')
cursor = conn.cursor()'''
for i in range(len(col1)):
    # ignore the column names
    if i == 0:
        continue
    email = col1[i]
    password = col2[i]
    # hash password to MD5 (32-bit Hex)
    password = hashlib.md5(password.encode()).hexdigest()
    sql = "INSERT IGNORE INTO Users (email,password) VALUES (%s,%s)"
    val = (email, password)
    cursor.execute(sql, val)
    conn.commit()


In [5]:
# read values from Categories.csv
columns = ['parent_category', 'category_name']
data = pd.read_csv('Categories.csv', names=columns)
col1 = data['parent_category'].values
col2 = data['category_name'].values

# import Categories data from Categories.csv to table: Categories
for i in range(len(col1)):
    # ignore the column names
    if i==0:
        continue
    parent_category=col1[i]
    category_name=col2[i]
    # prepare the sql statement
    sql="INSERT IGNORE INTO Categories (parent_category,category_name) VALUES (%s,%s)"
    val=(parent_category,category_name)
    cursor.execute(sql,val)
    conn.commit()